# DPKE_NTRU_HPS2048509

Esse trabalho foi desenvolvido com base no documento "NTRU Algorithm Specifications And Supporting Documentation".

In [165]:
import random
import unittest

 criar a base de polinômios; $\mathbb{Z}$ é o polinômio de coeficientes inteiros, de forma que $\mathbb{Z}/2\mathbb{Z}$, $\mathbb{Z}/3\mathbb{Z}$, $\mathbb{Z}/q\mathbb{Z}$, são inteiros módulo 2, 3 e q respectivamente.

In [166]:
q=2048
Z.<xZ> = ZZ[]              # ZZ[x]
Z2.<xZ2> = Integers(2)[]   # (ZZ/2ZZ)[x]
Z3.<xZ3> = Integers(3)[]   # (ZZ/3ZZ)[x]
Zq.<xZq> = Integers(q)[]   # (ZZ/2048ZZ)[x]
# xZ = Z.gen()
# xZ.content()
print( xZ.parent())
print(xZ2.parent())
print(xZ3.parent())
print(xZq.parent())

Univariate Polynomial Ring in xZ over Integer Ring
Univariate Polynomial Ring in xZ2 over Ring of integers modulo 2 (using GF2X)
Univariate Polynomial Ring in xZ3 over Ring of integers modulo 3
Univariate Polynomial Ring in xZq over Ring of integers modulo 2048


confirmada a base do anel, implementamos a convolução cíclica, utilizando funções nativas do SageMath

In [167]:
n= 509
Phi_n = lambda x : (x^n-1)//(x-1)
Phi_1 = lambda x : (x - 1)
R.<xR> = Z.quotient(Phi_1(xZ) * Phi_n(xZ))      # ZZ[x] / (x^n-1)
S.<xS> = Z.quotient(Phi_n(xZ))                  # ZZ[x] / (x^(n-1) + x^(n-2) + ... + x + 1)
R3.<xR3> = Z3.quotient(Phi_1(xZ3) * Phi_n(xZ3)) # ZZ[x] / (3, x^n-1)
Rq.<xRq> = Zq.quotient(Phi_1(xZq) * Phi_n(xZq)) # ZZ[x] / (q, x^n-1)              
S2.<xS2> = Z2.quotient(Phi_n(xZ2))              # ZZ[x] / (2, x^(n-1) + x^(n-2) + ... + x + 1)
S3.<xS3> = Z3.quotient(Phi_n(xZ3))              # ZZ[x] / (3, x^(n-1) + x^(n-2) + ... + x + 1)
Sq.<xSq> = Zq.quotient(Phi_n(xZq))              # ZZ[x] / (q, x^(n-1) + x^(n-2) + ... + x + 1)
# R1.<xR1> = Zq.quotient(Phi_1(xZq))

print("R:", xR.parent())
print("S:", xS.parent())
print("R3:", xR3.parent())
print("Rq:", xRq.parent()) 
print("S2:", xS2.parent())
print("S3:", xS3.parent())
print("Sq:", xSq.parent())
# print("R1:", xR1.parent())

R: Univariate Quotient Polynomial Ring in xR over Integer Ring with modulus xZ^509 - 1
S: Univariate Quotient Polynomial Ring in xS over Integer Ring with modulus xZ^508 + xZ^507 + xZ^506 + xZ^505 + xZ^504 + xZ^503 + xZ^502 + xZ^501 + xZ^500 + xZ^499 + xZ^498 + xZ^497 + xZ^496 + xZ^495 + xZ^494 + xZ^493 + xZ^492 + xZ^491 + xZ^490 + xZ^489 + xZ^488 + xZ^487 + xZ^486 + xZ^485 + xZ^484 + xZ^483 + xZ^482 + xZ^481 + xZ^480 + xZ^479 + xZ^478 + xZ^477 + xZ^476 + xZ^475 + xZ^474 + xZ^473 + xZ^472 + xZ^471 + xZ^470 + xZ^469 + xZ^468 + xZ^467 + xZ^466 + xZ^465 + xZ^464 + xZ^463 + xZ^462 + xZ^461 + xZ^460 + xZ^459 + xZ^458 + xZ^457 + xZ^456 + xZ^455 + xZ^454 + xZ^453 + xZ^452 + xZ^451 + xZ^450 + xZ^449 + xZ^448 + xZ^447 + xZ^446 + xZ^445 + xZ^444 + xZ^443 + xZ^442 + xZ^441 + xZ^440 + xZ^439 + xZ^438 + xZ^437 + xZ^436 + xZ^435 + xZ^434 + xZ^433 + xZ^432 + xZ^431 + xZ^430 + xZ^429 + xZ^428 + xZ^427 + xZ^426 + xZ^425 + xZ^424 + xZ^423 + xZ^422 + xZ^421 + xZ^420 + xZ^419 + xZ^418 + xZ^417 + xZ^416 + 

O resultado comprova que tanto o grau quanto as reduções de cada anel estão corretos. Entretanto de acordo com o artigo $Rq$, $Sq$ e $S3$ possuem coeficientes centralizados,  assim o resultado até aqui garante que ainda não foi obtida a representação correta.

#### Representantes canônicos

O Sage por padrão gera representantes canônicos com coeficientes dentro de intervalo não negativo $[0, q-1]$, não há uma função nativa que possa mudar esse comportamento. Os representantes canônicos definidos no artigo são específicos para o NTRU. Dessa forma foi implementada a função canonicalNTRU para converter coeficientes para o intervalo centrado.
$$
\begin{align*}
    \underline{R_q} \quad & \text{com grau } n-1 \text{ coeficientes dentro do intervalo } \left[ -q/2, q/2-1\right]. \\
    \underline{S_q} \quad & \text{com grau } n-2 \text{ coeficientes dentro do intervalo } \left[-q/2, q/2-1\right]. \\
    \underline{S3} \quad & \text{com grau } n-2 \text{ coeficientes em } \{-1, 0, 1\}.
\end{align*}
$$

In [168]:
def canonicalNTRU(ring, coeffs, n, q):
    a = ring(coeffs)

    if ring == S3:
        # n = n
        coeffs_can = twos_complement_S3(a, n)
    else:
        n = n-1
        coeffs_can = twos_complement_coeffs(a, n, q)

    Z.<z> = ZZ[]
    return Z(coeffs_can)

def twos_complement_S3(a, n):
    coeffs = [0] * n

    for i, ai in enumerate(a):
        coeffs[i % n] += int(ai)

    def rep_S3(c):
        r = c % 3
        if r == 2:
            return -1
        return r

    return [rep_S3(c) for c in coeffs]

def twos_complement_coeffs(a, n, q):
    coeffs = [0]*n
    for i, ai in enumerate(a):
        if i < n:
            coeffs[i] += int(ai)

    def rep_centered(val):
        r = int(val) % q
        if r > q//2 or (q % 2 == 0 and r == q//2):
            r = r - q
        return r

    return [rep_centered(c) for c in coeffs]
    

O trecho abaixo busca validar a função canonicalNTRU. Também busca elucidar como gerar polinômios através do sistema algébrico computacional do SageMath. Assim obtemos o representante canônico de acordo com as especificações.

 Apesar de assumirmos que estamos lidando com o representante canônico, o SageMath entende que o polinômio retornado pela função canonical não pertence mais ao anel do polinômio de entrada da função. Para que fosse possível manter a coerção seria necessário inicializar todas os polinômios em pase $\mathbb{Z}[\mathbf{x}]$ sem redução, realizando as reduções "manualmente"?

In [169]:
# criar polinômio no sage a partir de um vetor de coeficientes
P.<x> = ZZ['x'] # todo polinômio deve pertencer a um Anel de Polinômios
coeffs = [-2048, -1025, 1024, 2048]
p = P(coeffs)
print("p: ", P(coeffs))
print ("p em S3: ", Sq(p.list()))
print("p canonical representative: ", canonicalNTRU(Sq, p.list(), n, q))

p:  2048*x^3 + 1024*x^2 - 1025*x - 2048
p em S3:  1024*xSq^2 + 1023*xSq
p canonical representative:  -1024*z^2 + 1023*z


#### Sample $\mathcal{T}$ and $\mathcal{T(d)}$

Dado $g$ amostrado de $\mathcal{L}_g=\mathcal{T}$($d$), dessa forma definimos $d=\frac{q}{8}-2$
Onde $0 < d \leq \frac{2n}{3}$. Nesse contexto $d/2$ será o valor que define a quantidade de 1 e -1, que devem ser a mesma quantidade, o resto dos coeficientes será zero. Dessa forma definimos as condições para $q$ que deve ser potência de 2, consequentemente, inteiro e par. garante que $gcd(p,q)=1$. Após definir $n$ podemos definir $q$ a partir da desigualdade;
\begin{equation}
    \frac{q}{8}-2 \leq \frac{2n}{3} \quad\longrightarrow \quad q \leq 8 \bigg(\frac{2n}{3}+2\bigg)
\end{equation}
pode escolher até a maior potência de 2 menor ou igual a esse limite. Caso ainda haja interesse em usar um $q$ maior, a especificação recomenda fixar $d$ em $2 \lfloor n/3\rfloor$. $q$ aumenta o tamanho da chave, impactando o trade-off entre segurança e eficiência.

Definimos os polinômios ternários $\mathcal{T}$ e $\mathcal{T}(d)$, que serão usados tanto na geração das chaves $f$ e $g$, quanto para $r$ e $m$;
\begin{align*}
    \mathcal{T} \quad & \text{com grau } n-2 \text{ coeficientes em } \{-1, 0, 1\}. \\
    \mathcal{T}(d) \quad & \text{com grau } n-2 \text{ coeficientes em } \{-1, 0, 1\}. \\
\end{align*}

De acordo com as específicações do hps2048509 adotamos $p=3$, e $n=509$, consequentemente $q=2048$ sendo a maior potência de $2$ abaixo do limite em (1). Portanto $\mathcal{T}(d)$ deve ter 127 coeficientes 1's e 127 coeficientes -1's, restando 254 zeros.

In [170]:
def ternary(S,n):
    return  S(random.choices([-1, 0, 1], k=n-1))

def ternary_fixed(S,n,q):
    d = q//8-2
    ind = list(range(n-1))
    random.shuffle(ind)
    pos = ind[0:d//2]
    neg = ind[d//2:d]
    c = [0]*(n-1)

    for i in pos:
        c[i] = 1
    for i in neg:
        c[i] = -1
    return S(c)

In [171]:
def count(poly):
    coeffs = poly.list()
    zeros = coeffs.count(0)
    ones = coeffs.count(1)
    neg_ones = coeffs.count(-1)
    return zeros, ones, neg_ones

In [172]:
g = ternary_fixed(S, n, q)
print("g: ", g)
print("g_in_S3", canonicalNTRU(S3, g.list(), n, q))

zeros, ones, neg_ones = count(g)

print(f"Zeros: {zeros}")
print(f"Uns: {ones}")
print(f"Menos Uns: {neg_ones}")

g:  -xS^506 + xS^505 + xS^504 - xS^503 - xS^500 - xS^499 + xS^497 - xS^494 - xS^491 - xS^487 + xS^484 + xS^483 - xS^482 - xS^479 - xS^477 - xS^475 + xS^472 - xS^467 + xS^465 + xS^464 + xS^461 + xS^459 - xS^450 - xS^449 - xS^446 + xS^441 + xS^440 + xS^439 + xS^438 - xS^436 - xS^434 - xS^431 - xS^430 - xS^428 + xS^427 - xS^424 - xS^422 - xS^420 + xS^419 + xS^418 + xS^417 + xS^416 + xS^414 - xS^410 + xS^408 - xS^406 - xS^403 + xS^402 + xS^400 - xS^398 - xS^397 - xS^396 + xS^395 + xS^394 - xS^390 + xS^389 + xS^387 - xS^385 + xS^381 + xS^380 - xS^379 + xS^378 - xS^377 - xS^372 - xS^371 - xS^368 - xS^367 - xS^365 + xS^364 + xS^363 + xS^362 + xS^361 - xS^359 + xS^357 - xS^356 - xS^355 - xS^354 + xS^352 - xS^351 + xS^347 + xS^346 + xS^344 + xS^343 + xS^341 + xS^339 - xS^336 + xS^335 - xS^332 - xS^331 - xS^329 - xS^328 + xS^325 + xS^324 - xS^323 - xS^319 - xS^315 - xS^312 - xS^310 + xS^309 + xS^308 - xS^307 - xS^303 - xS^301 + xS^300 - xS^298 + xS^296 - xS^294 - xS^293 + xS^292 + xS^290 - xS^28

#### Inversa

O artigo não específica o método adotado para obter a inversa, neste caso é utilizado a técnica Hensel lifiting

In [173]:
def exponentiation(base, exp):
    result = 1
    current_power = base
    current_exponent = exp

    while current_exponent > 0:
        if current_exponent % 2 == 1:
            result *= current_power
        current_power *= current_power
        current_exponent //= 2
    return result

def S2_inverse(S2, a):
    # aa = S2(a.lift())^(2**508 - 2)
    aa = exponentiation(S2(a.lift()), 2**507 - 1)
    aa = aa^2
    return aa

def Sq_inverse(S2, Sq, a):
    # v0 = S2(a.lift())^(-1)
    v0 = S2_inverse(S2, a)
    v0 = Sq(v0.lift())
    for i in range(4):
        v0 = Sq(v0 * (2 - a * v0))
    return v0

## Geração de chaves (KeyGen)

O cálculo da inversa é necessário apenas na etapa de geração de chaves, como mostrado em "DPKE_Public_Key". 

O DPKE_Public_Key chama a função sample_fg para amostrar os polinômios $f$ e $g \in S$
\begin{align*}
    f \quad & \text{com grau } n-2 \text{ coeficientes em } \{-1, 0, 1\}. \\
    g \quad & \text{com grau } n-2 \text{ coeficientes em } \{-1, 0, 1\}. \\
\end{align*}

In [174]:
#section 1.10.1 (pg. 13)
def sample_fg(S, n, q):
    return ternary(S,n), ternary_fixed(S,n,q)

A partir das saidas podemos verificar, nesse caso como p é primo, fp é um corpo de galóis, ou seja sempre haverá inversa nesse anel, permitindo utilizar a função nativa do Sage

In [ ]:
def DPKE_Key_Pair(S, S2, S3, Sq, Rq, n, q):
    for attempt in range(5):
        f, g = sample_fg(S, n, q)
        # fp = canonicalNTRU(S3, f.list(), n, q)
        fp = 1 / S3(f.lift())
        h, hq, v0, v1 = DPKE_Public_Key(S, S2, Sq, Rq, n, q, f, g)

        print(f"[n,q]=[{n},{q}], attempt: {attempt + 1}")

        return f, g, fp, h, hq, v0, v1

In [176]:
def DPKE_Public_Key(S, S2, Sq, Rq, n, q, f, g):
        try:
            G = S(3) * g
            v0 = Sq((G * f).lift()) 
            v1 = Sq_inverse(S2, Sq, v0)
            G_Sq = Sq(G.lift())
            h_Sq = v1 * G_Sq * G_Sq
            h = Rq(h_Sq.lift())
            f_Sq = Sq(f.lift())
            hq = v1 * f_Sq * f_Sq

            return h, hq, v0, v1

        except (ZeroDivisionError, ArithmeticError):
            pass

        raise ValueError("limite de tentativas em keygen()")

In [177]:
f, g, fp, h, hq, v0, v1 = DPKE_Key_Pair(S, S2, S3, Sq, Rq, n, q)

print("f:",  f, "\n")
print("g:",  g, "\n")
print("fp:", fp,"\n")
print("h:",  h, "\n")
print("hq:", hq,"\n")
print("v0:", v0,"\n")
print("v1:", v1,"\n")

[n,q]=[509,2048], attempt: 1
f: -xS^505 + xS^504 - xS^502 - xS^499 - xS^498 + xS^497 - xS^496 + xS^495 - xS^494 + xS^493 - xS^492 - xS^491 + xS^490 - xS^487 + xS^486 + xS^485 + xS^482 - xS^481 - xS^480 + xS^479 - xS^478 - xS^476 + xS^474 + xS^473 + xS^469 - xS^468 + xS^466 - xS^465 - xS^464 + xS^463 + xS^462 + xS^461 + xS^460 - xS^459 + xS^458 - xS^457 - xS^455 + xS^452 - xS^450 - xS^449 - xS^448 - xS^447 - xS^446 - xS^445 + xS^444 - xS^443 - xS^439 + xS^438 - xS^437 - xS^436 - xS^435 - xS^434 - xS^433 - xS^432 - xS^431 + xS^428 + xS^426 - xS^425 + xS^424 - xS^423 + xS^422 - xS^420 + xS^419 + xS^417 + xS^415 - xS^414 + xS^413 - xS^412 + xS^411 + xS^410 - xS^408 - xS^406 - xS^405 - xS^403 - xS^401 - xS^399 - xS^398 + xS^397 + xS^396 - xS^395 + xS^394 - xS^392 + xS^391 - xS^389 - xS^387 - xS^386 + xS^385 + xS^384 - xS^383 + xS^377 + xS^376 + xS^375 + xS^374 + xS^373 - xS^371 - xS^369 - xS^368 - xS^367 - xS^366 - xS^364 - xS^362 - xS^360 - xS^359 - xS^358 - xS^357 - xS^356 - xS^355 - xS^3

## Encrypt

In [178]:
def sample_rm(S, n, q):
    return ternary(S,n), ternary_fixed(S,n,q) 

In [179]:
def DPKE_Encrypt(Rq, h, r, m):
    rq = Rq(r.lift())
    mq = Rq(m.lift())
    c = Rq(((rq*h).lift())+ mq)
    return c

## Decrypt

In [ ]:
def decrypt(S3, Sq, Rq, c, f, fp, hq, q):
    v1 = Rq((c*Rq(f.lift())).lift())
    m0 = S3((S3(v1.lift())*S3(fp.lift())).lift())
    m1 = m0.lift()
    r = Sq((Sq((c-Rq(m1)).lift())*hq).lift())
    r_coeffs = r.lift().list()
    m0_coeffs = m0.lift().list()
    
    r_ok = all(c in {-1, 0, 1} for c in r_coeffs) and (r != 0)
    
    weight = q // 8 - 2
    m0_ok = (all(c in {-1, 0, 1} for c in m0_coeffs) and 
             (m0 != 0) and
             sum(1 for c in m0_coeffs if c == 1) == weight // 2 and
             sum(1 for c in m0_coeffs if c == -1) == weight // 2)
    fail = 0 if (r_ok and m0_ok) else 1
    print(fail)
    return r, m0, fail


## Testes e profiling

In [181]:

class TestNTRU(unittest.TestCase):
    def setUp(self):
        self.qs = {}
        self.rings = {}
        self.keys = {}
        self.qs[509] = 2048
        self.ns = [509]
        self.p = 3
        
        for n in self.ns:
            q = self.qs[n]
            Z.<xZ> = ZZ[]                                   # ZZ[x]
            Z2.<xZ2> = Integers(2)[]                        # (ZZ/2ZZ)[x]
            Z3.<xZ3> = Integers(3)[]                        # (ZZ/3ZZ)[x]
            Zq.<xZq> = Integers(q)[]                        # (ZZ/2048ZZ)[x]
            Phi_n = lambda x : (x^n-1)//(x-1)
            Phi_1 = lambda x : (x - 1)
            R.<xR> = Z.quotient(Phi_1(xZ) * Phi_n(xZ))      # ZZ[x] / (x^n-1)
            S.<xS> = Z.quotient(Phi_n(xZ))                  # ZZ[x] / (x^(n-1) + x^(n-2) + ... + x + 1)
            R3.<xR3> = Z3.quotient(Phi_1(xZ3) * Phi_n(xZ3)) # ZZ[x] / (3, x^n-1)
            Rq.<xRq> = Zq.quotient(Phi_1(xZq) * Phi_n(xZq)) # ZZ[x] / (q, x^n-1)              
            S2.<xS2> = Z2.quotient(Phi_n(xZ2))              # ZZ[x] / (2, x^(n-1) + x^(n-2) + ... + x + 1)
            S3.<xS3> = Z3.quotient(Phi_n(xZ3))              # ZZ[x] / (3, x^(n-1) + x^(n-2) + ... + x + 1)
            Sq.<xSq> = Zq.quotient(Phi_n(xZq))              # ZZ[x] / (q, x^(n-1) + x^(n-2) + ... + x + 1)
            R1.<xR1> = Zq.quotient(Phi_1(xZq))            

            self.rings[(n, q)] = {
                'S'    :  S,
                'Sq'   :  Sq,
                'Rq'   :  Rq,
                'S2'   :  S2,
                'S3'   :  S3,
                'R'    :  R,
                'Phi_n':  Phi_n,
                'R1'   :  R1
            }
            rings = self.rings[(n,q)]

            h, hq, v0, v1 = DPKE_Public_Key(rings['S'], rings['S2'],
                                    rings['Sq'], rings['Rq'], n, q, f, g)
            
            self.keys[(n,q)] = {
                'f'  : f,
                'g'  : g,
                'fp' : fp,
                'h'  : h,
                'hq' : hq
            }

teste do DPKE_Public_Key

In [182]:

class TestKeygen(TestNTRU):
    def test_parameters(self):
        for n in self.ns:
            q = self.qs[n]
            d = q//8 - 2
            self.assertTrue((q & (q - 1)) == 0)     # q is a power of 2: (2^n = q)
            self.assertGreaterEqual(q, 32)          # q is not small?
            self.assertLessEqual(d, 2*n/3)          # limit of d
            self.assertEqual(gcd(self.p, q), 1)     # p and q are coprime
            self.assertLessEqual(self.p, q)         # p equal or less q 

    def test_ternary(self):
        for n in self.ns:
            q = self.qs[n]
            rings = self.rings[(n,q)]

            for i in range(1):
                t = ternary(rings['S'], n)

                t_lift = t.lift()
                self.assertLessEqual(t_lift.degree(), n-2)                      # t of degree at most n-2 (item)              
                self.assertTrue(all(coef in [-1, 0, 1] for coef in t_lift.list()))    # t is ternary polinomial
                self.assertNotEqual(t, 0)                                       # t non-zero ternary polinomial
                self.assertIs(t.parent(), rings['S'])                           # t is an element of S

    def test_ternary_fixed(self):
        for n in self.ns:
            q = self.qs[n]
            rings = self.rings[(n,q)]

            for i in range(1):
                t_fixed = ternary_fixed(rings['S'], n, q)
                
                t_fixed_lift = t_fixed.lift()
                self.assertLessEqual(t_fixed_lift.degree(), n-2)                          # t_fixed of degree at most n-2
                self.assertTrue(all(coef in [-1, 0, 1] for coef in t_fixed_lift.list()))        # t_fixed is ternary polynomial
                self.assertNotEqual(t_fixed, 0)                                           # t_fixed non-zero ternary polinomial
                self.assertEqual(sum(1 for coef in t_fixed_lift.list() if coef == 1), q//16-1)  # t_fixed have exactly (q//16-1) coefficients equal to +1
                self.assertEqual(sum(1 for coef in t_fixed_lift.list() if coef == -1), q//16-1) # t_fixed have exactly (q//16-1) coefficients equal to -1
                self.assertIs(t_fixed.parent(), rings['S'])                               # t_fixed is an element of S           

    def test_sample_fg(self):
        for n in self.ns:
            q = self.qs[n]
            rings = self.rings[(n, q)]

            for i in range(1):
                f, g = sample_fg(rings['S'], n, q)

                f_lift = f.lift()
                self.assertLessEqual(f_lift.degree(), n-2)                          # f of degree at most n-2 (item)              
                self.assertTrue(all(coef in [-1, 0, 1] for coef in f_lift.list()))        # f is ternary polinomial
                self.assertNotEqual(f, 0)                                           # f non-zero ternary polinomial
                self.assertIs(f.parent(), rings['S'])                               # f is an element of S
                
                g_lift = g.lift()
                self.assertLessEqual(g_lift.degree(), n-2)                          # g of degree at most n-2
                self.assertTrue(all(coef in [-1, 0, 1] for coef in g_lift.list()))        # g is ternary polynomial
                self.assertNotEqual(g, 0)                                           # g non-zero ternary polinomial
                self.assertEqual(sum(1 for coef in g_lift.list() if coef == 1), q//16-1)  # g have exactly (q//16-1) coefficients equal to +1
                self.assertEqual(sum(1 for coef in g_lift.list() if coef == -1), q//16-1) # g have exactly (q//16-1) coefficients equal to -1
                self.assertIs(g.parent(), rings['S'])                               # g is an element of S

    def test_polynomial_ring_keygen(self):
        for n in self.ns:
            q = self.qs[n]
            rings = self.rings[(n, q)]
            keys = self.keys[(n, q)]
            
            G = 3 * keys['g']
            v0 = rings['Sq']((G * keys['f']).lift())
            v1 = Sq_inverse(rings['S2'], rings['Sq'], v0)

            self.assertEqual(G.parent(), rings['S'])
            self.assertEqual(v0.parent(), rings['Sq'])
            self.assertEqual(keys['h'].parent(), rings['Rq'])
            self.assertEqual(v1.parent(), rings['Sq'])
            self.assertEqual(keys['hq'].parent(), rings['Sq'])

            self.assertEqual(keys['h'], rings['Rq']((v1 * rings['Sq']((G * G).lift())).lift()))
            self.assertEqual(keys['hq'], rings['Sq']((v1 * rings['Sq']((keys['f'] * keys['f']).lift())).lift()))

            R1 = rings['R1']
            self.assertEqual(R1(keys['g'].lift()), 0) # output section 1.11.2 (notes) 
            # 1.11.2 - Output condition
            # self.assertEqual((keys['h']*rings['Rq'](keys['f'])).lift(), Rq((3*keys['g'])).lift())
            
    def test_inverse(self):
        for n in self.ns:
            q = self.qs[n]
            rings = self.rings[(n, q)]

            for _ in range(1):
                h, hq, v0, v1 = DPKE_Public_Key(rings['S'], rings['S2'],
                                        rings['Sq'], rings['Rq'], n, q, f, g)

                f_in_S3 = rings['S3'](f.lift())

                self.assertEqual(f_in_S3 * fp, 1)
    
                h_Sq = rings['Sq'](h.lift())
                self.assertEqual(h_Sq * hq, 1)

                self.assertEqual(v0 * v1, 1)

unittest.main(argv=[''], exit=False)
t  = TestKeygen()
t.setUp() 
print("Profiling test_parameters:")
%prun -l 10 t.test_parameters()
t.setUp() 
print("Profiling test_ternary:")
%prun -l 10 t.test_ternary()
t.setUp() 
print("Profiling test_ternary_fixed:")
%prun -l 10 t.test_ternary_fixed() 
print("\nProfiling test_sample_fg:")
t.setUp()
%prun -l 20 t.test_sample_fg() 
print("\nProfiling test_inverse:")
t.setUp()
%prun -l 30 t.test_inverse()

E...E...E.....
ERROR: test_DPKE_Decrypt_op (__main__.TestDPKE_Decrypt.test_DPKE_Decrypt_op)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/tmp/ipykernel_16768/1130661994.py", line 35, in test_DPKE_Decrypt_op
    r, m0, fail = DPKE_Decrypt(rings['S3'], rings['Sq'], rings['Rq'], c, keys['f'], keys['fp'], keys['hq'], q)
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_16768/1796860440.py", line 3, in DPKE_Decrypt
    m0 = S3((S3(v1.lift())*S3(fp.list())).lift())
                              ^^^^^^^
  File "sage/structure/element.pyx", line 495, in sage.structure.element.Element.__getattr__ (build/cythonized/sage/structure/element.c:13068)
    return self.getattr_from_category(name)
  File "sage/structure/element.pyx", line 508, in sage.structure.element.Element.getattr_from_category (build/cythonized/sage/structure/element.c:13178)
    r

Profiling test_parameters:
 Profiling test_ternary:
 Profiling test_ternary_fixed:
 
Profiling test_sample_fg:
 
Profiling test_inverse:


TypeError: unsupported operand parent(s) for *: 'Univariate Quotient Polynomial Ring in xS3 over Ring of integers modulo 3 with modulus xZ3^508 + xZ3^507 + xZ3^506 + xZ3^505 + xZ3^504 + xZ3^503 + xZ3^502 + xZ3^501 + xZ3^500 + xZ3^499 + xZ3^498 + xZ3^497 + xZ3^496 + xZ3^495 + xZ3^494 + xZ3^493 + xZ3^492 + xZ3^491 + xZ3^490 + xZ3^489 + xZ3^488 + xZ3^487 + xZ3^486 + xZ3^485 + xZ3^484 + xZ3^483 + xZ3^482 + xZ3^481 + xZ3^480 + xZ3^479 + xZ3^478 + xZ3^477 + xZ3^476 + xZ3^475 + xZ3^474 + xZ3^473 + xZ3^472 + xZ3^471 + xZ3^470 + xZ3^469 + xZ3^468 + xZ3^467 + xZ3^466 + xZ3^465 + xZ3^464 + xZ3^463 + xZ3^462 + xZ3^461 + xZ3^460 + xZ3^459 + xZ3^458 + xZ3^457 + xZ3^456 + xZ3^455 + xZ3^454 + xZ3^453 + xZ3^452 + xZ3^451 + xZ3^450 + xZ3^449 + xZ3^448 + xZ3^447 + xZ3^446 + xZ3^445 + xZ3^444 + xZ3^443 + xZ3^442 + xZ3^441 + xZ3^440 + xZ3^439 + xZ3^438 + xZ3^437 + xZ3^436 + xZ3^435 + xZ3^434 + xZ3^433 + xZ3^432 + xZ3^431 + xZ3^430 + xZ3^429 + xZ3^428 + xZ3^427 + xZ3^426 + xZ3^425 + xZ3^424 + xZ3^423 + xZ3^

         4067 function calls in 0.003 seconds

   Ordered by: internal time
   List reduced from 40 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.002    0.002 1314150491.py:42(test_sample_fg)
      509    0.000    0.000    0.000    0.000 1314150491.py:52(<genexpr>)
      509    0.000    0.000    0.000    0.000 1314150491.py:58(<genexpr>)
      507    0.000    0.000    0.000    0.000 random.py:242(_randbelow_with_getrandbits)
        1    0.000    0.000    0.001    0.001 2170615749.py:4(ternary_fixed)
      128    0.000    0.000    0.000    0.000 1314150491.py:61(<genexpr>)
        1    0.000    0.000    0.000    0.000 random.py:454(choices)
        1    0.000    0.000    0.000    0.000 random.py:350(shuffle)
      128    0.000    0.000    0.000    0.000 1314150491.py:60(<genexpr>)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
        2    0.000    0

test DPKE_Encrypt

In [ ]:
class TestDPKE_Encrypt(TestNTRU):
    def test_sample_rm(self):
        for n in self.ns:
            q = self.qs[n]
            rings = self.rings[(n, q)]

            for i in range(1):
                r, m = sample_rm(rings['S'], n, q)

                r_lift = r.lift()
                self.assertLessEqual(r_lift.degree(), n-2)                          # r of degree at most n-2 (item)              
                self.assertTrue(all(coef in [-1, 0, 1] for coef in r_lift.list()))        # r is ternary polinomial
                self.assertNotEqual(r, 0)                                           # r non-zero ternary polinomial
                self.assertIs(r.parent(), rings['S'])                               # r is an element of S
                
                m_lift = m.lift()
                self.assertLessEqual(m_lift.degree(), n-2)                          # m of degree at most n-2
                self.assertTrue(all(coef in [-1, 0, 1] for coef in m_lift.list()))        # m is ternary polynomial
                self.assertNotEqual(m, 0)                                           # m non-zero ternary polinomial
                self.assertEqual(sum(1 for coef in m_lift.list() if coef == 1), q//16-1)  # m have exactly (q//16-1) coefficients equal to +1
                self.assertEqual(sum(1 for coef in m_lift.list() if coef == -1), q//16-1) # m have exactly (q//16-1) coefficients equal to -1
                self.assertIs(m.parent(), rings['S'])                               # m is an element of S
        
    def test_operations(self):
        for n in self.ns:
            q = self.qs[n]
            rings = self.rings[(n,q)]
            keys = self.keys[(n,q)]

            for _ in range(1):
                r, m = sample_fg(rings['S'], n, q)
                c = DPKE_Encrypt(rings['Rq'], keys['h'], r, m)
                self.assertIs(c.parent(), rings['Rq']) # c is an element of Rq

t1 = TestDPKE_Encrypt()
t1.setUp() 
print("Profiling test_operations:")
%prun -l 10 t1.test_operations()

Profiling test_operations:
 

         2822 function calls in 0.002 seconds

   Ordered by: internal time
   List reduced from 27 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 random.py:454(choices)
      507    0.000    0.000    0.000    0.000 random.py:242(_randbelow_with_getrandbits)
        1    0.000    0.000    0.001    0.001 2170615749.py:4(ternary_fixed)
        7    0.000    0.000    0.000    0.000 polynomial_quotient_ring_element.py:108(__init__)
        1    0.000    0.000    0.000    0.000 random.py:350(shuffle)
        4    0.000    0.000    0.000    0.000 polynomial_ring.py:335(_element_constructor_)
        1    0.000    0.000    0.001    0.001 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
      709    0.000    0.000    0.000    0.000 {method 'getrandbits' of '_random.Random' objects}
        5    0.000    0.000    0.00

In [ ]:
class TestDPKE_Decrypt(TestNTRU):
    def test_sample_rm(self):
        for n in self.ns:
            q = self.qs[n]
            rings = self.rings[(n, q)]

            for i in range(1):
                r, m = sample_rm(rings['S'], n, q)

                r_lift = r.lift()
                self.assertLessEqual(r_lift.degree(), n-2)                          # r of degree at most n-2 (item)              
                self.assertTrue(all(coef in [-1, 0, 1] for coef in r_lift.list()))        # r is ternary polinomial
                self.assertNotEqual(r, 0)                                           # r non-zero ternary polinomial
                self.assertIs(r.parent(), rings['S'])                               # r is an element of S
                
                m_lift = m.lift()
                self.assertLessEqual(m_lift.degree(), n-2)                          # m of degree at most n-2
                self.assertTrue(all(coef in [-1, 0, 1] for coef in m_lift.list()))        # m is ternary polynomial
                self.assertNotEqual(m, 0)                                           # m non-zero ternary polinomial
                self.assertEqual(sum(1 for coef in m_lift.list() if coef == 1), q//16-1)  # m have exactly (q//16-1) coefficients equal to +1
                self.assertEqual(sum(1 for coef in m_lift.list() if coef == -1), q//16-1) # m have exactly (q//16-1) coefficients equal to -1
                self.assertIs(m.parent(), rings['S'])                               # m is an element of S

    def test_DPKE_Decrypt_op(self):
        for n in self.ns:
            q = self.qs[n]
            rings = self.rings[(n,q)]
            keys = self.keys[(n,q)]

            for _ in range(1):
                r, m = sample_fg(rings['S'], n, q)
                c = DPKE_Encrypt(rings['Rq'], keys['h'], r, m)
                self.assertIs(c.parent(), rings['Rq']) # c is an element of Rq
                
                r, m0, fail = DPKE_Decrypt(rings['S3'], rings['Sq'], rings['Rq'], c, keys['f'], keys['fp'], keys['hq'], q)
                print(fail) #
                print("r:", r)
                print("m0:", m0)